In [1]:
import numpy as np

In [33]:
from sklearn.datasets import load_svmlight_file
from scipy.sparse import csr_matrix
import numpy as np

# Supported files:
# 1. a9a.txt
# 2. w8a.txt
# 3. ijcnn1
# 4. covtype.libsvm.binary

def data_loader(file_name='a9a.txt'):
    # Load in data and get rid of last column
    X, y = load_svmlight_file(file_name)
    
    # Convert to numpy array
    X = X.todense()
    return X, y



In [34]:
X, y = data_loader('covtype.libsvm.binary')

print(X.shape)

(581012, 54)


In [ ]:
# if we want to use pytorch to calculate the sgd of f(x)
# class f(torch.nn.Module):

In [2]:
def lagrangian(x_k, y_k1, y_j, y_k, z_k): pass

def lagrangian_hat(x, y_k1, z_k, v_k): pass

In [ ]:
def reg_vk(k, q, ...):
    # use some sgd library?
    if k % q == 0:
        vk = ...
    else:
        vk = ...

    return vk

# implement later
# def online_vk(k, q, ...):
#     if k % q == 0:
#         vk = ...
#     else:
#         vk = ...

#     return vk

In [ ]:
# Let b1 = None unless performing online algo (rename if necessary)
def spider_admm(b1, b2, q, K, n, p):
    # init
    x_k = ...
    j = ...
    y_k = ...
    z_k = ...
    
    for k in range(K):
        if b1 is None: 
            vk = reg_vk(k, q, ...)
        else: 
            # implement later
            # vk = online_vk(k, q, ...)
        
        # update k + 1 for y, x, z
        # probably do not need separate k + 1 variables

        # using lagrangian
        lagrs = []
        for j in range(m):
            y_k[j] = np.argmin(...)

        # using lagrangian hat
        lagr_hats = []
        x_k = np.argmin(...)

        z_k = ...

    return [x_k, y_k, z_k]

def online_spider_admm(b1, b2, q, K, n, p): 
    return spider_admm(b1, b2, q, K, n, p)